In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
# pandas global settings
pd.options.display.max_columns=200

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [11]:
path="../../../../Datafiles/LendingClub/"
file = 'LoanStats3d2015.csv.zip'
url = 'https://resources.lendingclub.com/LoanStats3d.csv.zip'
print("We are working with 2015 Lending dataset from {}".format(url))
df = pd.read_csv(path + file, header=2,compression='zip', sep=',', quotechar='"', skipfooter=2)
print("there are {:,} entries in the data frame, with {} columns, from data file: {}".format(len(df),
                                                                                             len(df.columns),
                                                                                             file))
y2015 = df

We are working with 2015 Lending dataset from https://resources.lendingclub.com/LoanStats3d.csv.zip


/Users/lou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """


there are 421,094 entries in the data frame, with 144 columns, from data file: LoanStats3d2015.csv.zip


In [12]:
for col in y2015.columns:
    print("column: {}".format(col))
    if col == 'id':
        print("********************* ID found dude ******************")

column: Unnamed: 0
column: Unnamed: 1
column: 24700
column: 24700.1
column: 24700.2
column:  36 months
column:  11.99%
column: 820.28
column: C
column: C1
column: Engineer
column: 10+ years
column: MORTGAGE
column: 65000
column: Not Verified
column: Dec-2015
column: Fully Paid
column: n
column: Unnamed: 18
column: Unnamed: 19
column: small_business
column: Business
column: 577xx
column: SD
column: 16.06
column: 1
column: Dec-1999
column: 4
column: 6
column: Unnamed: 29
column: 22
column: 0
column: 21470
column: 19.2%
column: 38
column: w
column: 0.00
column: 0.00.1
column: 25679.66
column: 25679.66.1
column: 24700.00
column: 979.66
column: 0.0
column: 0.0.1
column: 0.0.2
column: Jun-2016
column: 926.35
column: Unnamed: 47
column: Apr-2019
column: 0.1
column: Unnamed: 50
column: 1.1
column: Individual
column: Unnamed: 53
column: Unnamed: 54
column: Unnamed: 55
column: 0.2
column: 0.3
column: 204396
column: 1.2
column: 1.3
column: 0.4
column: 1.4
column: 19
column: 18005
column: 73
colum

In [13]:
y2015.head()

,Unnamed: 0,Unnamed: 1,24700,24700.1,24700.2,36 months,11.99%,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000,Not Verified,Dec-2015,Fully Paid,n,Unnamed: 18,Unnamed: 19,small_business,Business,577xx,SD,16.06,1,Dec-1999,4,6,Unnamed: 29,22,0,21470,19.2%,38,w,0.00,0.00.1,25679.66,25679.66.1,24700.00,979.66,0.0,0.0.1,0.0.2,Jun-2016,926.35,Unnamed: 47,Apr-2019,0.1,Unnamed: 50,1.1,Individual,Unnamed: 53,Unnamed: 54,Unnamed: 55,0.2,0.3,204396,1.2,1.3,0.4,1.4,19,18005,73,2,3,6472,29,111800,0.5,0.6,6.1,4.1,9733,57830,27.1,0.7,0.8,113,192,2.1,2.2,4.2,2.3,Unnamed: 86,0.9,6.2,0.10,5,5.1,13,17,6.3,20,27,5.2,22.1,0.11,0.12,0.13,2.4,97.4,7.7,0.14,0.15,314017,39475,79300,24667,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,N,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,N.1,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143
0,NaN,NaN,35000,35000,35000.0,60 months,14.85%,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,076xx,NJ,17.06,0,Sep-2008,0,NaN,NaN,13,0,7802,11.6%,17,w,14623.48,14623.48,33123.810000,33123.81,20376.52,12747.29,0.0,0.0,0.0,Apr-2019,829.90,May-2019,Apr-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,301500,1.0,1.0,0.0,1.0,23.0,12609.0,70.0,1.0,1.0,6987.0,45.0,67300,0.0,1.0,0.0,2,23192,54962.0,12.1,0,0,36.0,87,2,2,1,2.0,NaN,NaN,NaN,0,4,5,8,10,2,10,13.0,5,13,0.0,0,0,1,100.0,0.0,0,0,381215,52226,62500,18000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,3600,3600,3600.0,36 months,13.99%,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0,Aug-2003,1,30.0,NaN,7,0,2765,29.7%,13,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Apr-2019,0,30.0,1,Individual,NaN,NaN,NaN,0,722,144904,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300,3.0,1.0,4.0,4,20701,1506.0,37.2,0,0,148.0,128,3,3,1,4.0,69.0,4.0,69.0,2,2,4,2,5,3,4,9.0,4,7,0.0,0,0,3,76.9,0.0,0,0,178050,7746,2400,13734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,10400,10400,10400.0,60 months,22.45%,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,n,NaN,NaN,major_purchase,Major purchase,174xx,PA,25.37,1,Jun-1998,3,12.0,NaN,12,0,21929,64.5%,35,w,0.00,0.00,11740.500000,11740.50,10400.00,1340.50,0.0,0.0,0.0,Jul-2016,10128.96,NaN,Mar-2018,0,NaN,1,Individual,NaN,NaN,NaN,0,0,331730,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000,2.0,1.0,3.0,10,27644,4567.0,77.5,0,0,128.0,210,4,4,6,4.0,12.0,1.0,12.0,0,4,6,5,9,10,7,19.0,6,12,0.0,0,0,4,96.6,60.0,0,0,439570,95768,20300,88097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,20000,20000,20000.0,60 months,10.78%,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,n,NaN,NaN,home_improvement,NaN,605xx,IL,10.78,0,Aug-2000,0,NaN,NaN,6,0,7869,56.2%,18,w,0.00,0.00,22705.924294,22705.92,20000.00,2705.92,0.0,0.0,0.0,Jun-2017,15813.30,NaN,Mar-2019,0,NaN,1,Joint App,71000.0,13.85,Not Verified,0,0,189699,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000,2.0,5.0,1.0,6,31617,2737.0,55.9,0,0,125.0,184,14,14,5,101.0,NaN,10.0,NaN,0,2,3,2,4,6,4,7.0,3,6,0.0,0,0,0,100.0,50.0,0,0,218418,18696,6200,14877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,11950,11950,11950.0,36 months,13.44%,405.18,C,C3,Veterinary Tecnician,4 years,RENT,34000.0,Sourc

## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical colummn and see what happens off of this kind of naive approach.

In [28]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

KeyError: "['loan_status'] not found in axis"

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [14]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

 36 months
2
 11.99%
111
C
7
C1
35
Engineer
120812
10+ years
11
MORTGAGE
4
Not Verified
3
Dec-2015
12
Fully Paid
7
n
2
Unnamed: 19
34
small_business
14
Business
27
577xx
914
SD
49
Dec-1999
668
19.2%
1211
w
2
Jun-2016
52
Unnamed: 47
7
Apr-2019
53
Individual
2
Unnamed: 55
1
N
2
Unnamed: 123
1
Unnamed: 124
9
Unnamed: 125
3
Unnamed: 128
27
Unnamed: 129
28
Unnamed: 130
28
Unnamed: 133
4
N.1
2
Unnamed: 138
45
Unnamed: 139
3
Unnamed: 140
48


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [15]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

KeyError: 'id'

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [8]:
y2015.tail()

,Unnamed: 0,Unnamed: 1,24700,24700.1,24700.2,36 months,11.99%,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000,Not Verified,Dec-2015,Fully Paid,n,Unnamed: 18,Unnamed: 19,small_business,Business,577xx,SD,16.06,1,Dec-1999,4,6,Unnamed: 29,22,0,21470,19.2%,38,w,0.00,0.00.1,25679.66,25679.66.1,24700.00,979.66,0.0,0.0.1,0.0.2,Jun-2016,926.35,Unnamed: 47,Apr-2019,0.1,Unnamed: 50,1.1,Individual,Unnamed: 53,Unnamed: 54,Unnamed: 55,0.2,0.3,204396,1.2,1.3,0.4,1.4,19,18005,73,2,3,6472,29,111800,0.5,0.6,6.1,4.1,9733,57830,27.1,0.7,0.8,113,192,2.1,2.2,4.2,2.3,Unnamed: 86,0.9,6.2,0.10,5,5.1,13,17,6.3,20,27,5.2,22.1,0.11,0.12,0.13,2.4,97.4,7.7,0.14,0.15,314017,39475,79300,24667,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,N,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,N.1,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143
421089,NaN,NaN,10000,10000,10000.0,36 months,11.99%,332.10,B,B5,Office Assistant,8 years,RENT,31000.0,Verified,Jan-2015,Fully Paid,n,NaN,NaN,debt_consolidation,Debt consolidation,905xx,CA,28.69,0,Sep-2004,0,NaN,77.0,9,1,14037,82.1%,15,f,0.0,0.0,11947.244045,11947.24,10000.00,1947.24,0.0,0.00,0.0000,Jan-2018,331.90,NaN,Feb-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,25274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17100,NaN,NaN,NaN,2,3159,448.0,95.1,0,0,122.0,115,17,10,0,90.0,NaN,9.0,NaN,0,3,6,3,4,7,7,8.0,6,9,0.0,0,0,1,100.0,100.0,0,0,32950,25274,9200,15850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421090,NaN,NaN,24000,24000,24000.0,36 months,11.99%,797.03,B,B5,Director of Maintenance,10+ years,MORTGAGE,79000.0,Verified,Jan-2015,Fully Paid,n,NaN,NaN,home_improvement,Home improvement,072xx,NJ,3.90,0,Mar-1974,1,26.0,NaN,5,0,8621,84.5%,23,f,0.0,0.0,28663.686660,28663.69,24000.00,4663.69,0.0,0.00,0.0000,Feb-2018,2.58,NaN,Jan-2019,0,29.0,1,Individual,NaN,NaN,NaN,0,0,140285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10200,NaN,NaN,NaN,3,35071,976.0,89.2,0,0,NaN,488,3,3,4,3.0,29.0,3.0,29.0,10,2,3,3,14,0,4,17.0,3,5,0.0,0,0,2,56.5,100.0,0,0,152650,8621,9000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421091,NaN,NaN,13000,13000,13000.0,60 months,15.99%,316.07,D,D2,Radiologist Technologist,5 years,RENT,35000.0,Verified,Jan-2015,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,378xx,TN,30.90,0,Sep-2003,0,NaN,84.0,9,1,11031,61.3%,22,w,0.0,0.0,6109.520000,6109.52,2527.70,2506.32,0.0,1075.50,193.5900,May-2016,316.07,NaN,Dec-2016,0,NaN,1,Individual,NaN,NaN,NaN,0,0,34178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18000,NaN,NaN,NaN,7,3798,2554.0,75.9,0,0,134.0,108,6,6,0,17.0,NaN,10.0,NaN,0,4,6,4,4,12,7,10.0,6,9,0.0,0,0,3,100.0,50.0,1,0,51239,34178,10600,33239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421092,NaN,NaN,12000,12000,12000.0,60 months,19.99%,317.86,E,E3,Painter,1 year,RENT,64400.0,Source Verified,Jan-2015,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,010xx,MA,27.19,1,Oct-2003,2,22.0,NaN,17,0,8254,30.6%,20,w,0.0,0.0,5366.990000,5366.99,1552.69,2234.98,0.0,1579.32,284.2776,Jan-2016,317.86,NaN,Oct-2016,1,22.0,1,Individual,NaN,NaN,NaN,0,0,58418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27000,NaN,NaN,NaN,6,3895,3752.0,61.3,0,0,56.0,83,5,5,0,12.0,NaN,0.0,NaN,1,5,7,7,7,3,15,16.0,7,17,0.0,0,1,2,95.0,66.7,0,0,96919,58418,9700,69919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421093,NaN,NaN,20000,20000,20000.0,36 months,11.99%,664.20,B,B5,Manager Hotel Operations Oasis,10+ years,RENT,100000.0,Verified,Jan-2015,Fully Paid,n,NaN,Na

In [ ]:
# Remove two summary rows at the end that don't actually contain data.
# y2015 = y2015[:-2]
# skipfooter=2 in read_csv fixed this better...

Now this should be better. Let's try again.

In [ ]:
pd.get_dummies(y2015)

It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [ ]:
# Your code here.

